In [ ]:
!pip install numba

MATRIX-MATRIX MULTIPLICATION:

In [ ]:
import numpy as np
from numba import cuda, njit, float32
import time

# Constants for the number of threads per block in the GPU kernel
THREADS_PER_BLOCK_ROW = 32
THREADS_PER_BLOCK_COL = 32

@cuda.jit
def matrix_multiply_gpu(A, B, C):
    """
    GPU kernel for matrix multiplication using CUDA.

    Parameters:
    - A, B: Input matrices
    - C: Output matrix
    """
    ### ------------------- TO DO ------------------- ###

@njit
def naive_matrix_multiply_cpu(A, B, C):
    """
    Naive CPU implementation for matrix multiplication.

    Parameters:
    - A, B: Input matrices
    - C: Output matrix
    """
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            value = 0.0
            # Iterate over the shared dimension of A and B
            for k in range(A.shape[1]):
                value += A[i, k] * B[k, j]
            C[i, j] = value

def generate_random_square_matrix(size):
    """
    Generate a random square matrix of the specified size.

    Parameter:
    - size: Size of the square matrix

    Returns:
    - Random square matrix
    """
    return (np.random.rand(size, size) * 1000).astype(np.float32)

def check_equal(A1, A2):
    """
    Check if two matrices are equal within a specified tolerance.

    Parameters:
    - A1, A2: Matrices to compare

    Returns:
    - True if matrices are equal, False otherwise
    """
    return np.allclose(A1, A2, rtol=1e-5)

def main():
    # Set the matrix size (original size, not power of two)
    matrix_size = 10

    # Generate random square matrices A and B
    A = generate_random_square_matrix(1 << matrix_size)
    B = generate_random_square_matrix(1 << matrix_size)

    # Initialize output matrices C and C_cpu
    C = np.zeros_like(A)
    C_cpu = np.zeros_like(A)

    ### ------------------- TO DO 1------------------- ###

    # Perform matrix multiplication on the GPU
    start_time = time.time()
    ### ------------------- TO DO 2------------------- ###
    gpu_time = time.time() - start_time

    ### ------------------- TO DO 3------------------- ###

    # Perform matrix multiplication on the CPU
    start_time = time.time()
    naive_matrix_multiply_cpu(A, B, C_cpu)
    cpu_time = time.time() - start_time

    # Check if the CPU and GPU results are equal
    #if check_equal(C, C_cpu):
    #    print("\nCPU and GPU results are equal")
    #else:
    #    print("\nError! CPU and GPU results are different")

    # Print the execution times
    print(f"GPU Time: {gpu_time:.6f} seconds")
    print(f"CPU Time (naive): {cpu_time:.6f} seconds")

if __name__ == "__main__":
    main()

NORMALIZATION OF VECTORS:

In [ ]:
import numpy as np
from numba import cuda, float32
import math
import time

# Constants for the number of threads per block in the GPU kernel
THREADS_PER_BLOCK = 128

@cuda.jit
def vector_normalize_gpu(input_array, output_array):
    """
    GPU kernel for vector normalization using CUDA.

    Parameters:
    - input_array: Input vector
    - output_array: Normalized vector (output)
    """
    ### ------------------- TO DO ------------------- ###

def naive_vector_normalize_cpu(input_vector, output_vector):
    """
    Naive CPU implementation for vector normalization.

    Parameters:
    - input_vector: Input vector
    - output_vector: Normalized vector (output)
    """
    sum_of_squares = 0.0

    # Compute the sum of squares
    for value in input_vector:
        sum_of_squares += value ** 2

    # Normalize the vector using the total sum of squares
    for i in range(len(input_vector)):
        output_vector[i] = input_vector[i] / math.sqrt(sum_of_squares)

def generate_random_vector(size):
    """
    Generate a random vector of the specified size.

    Parameter:
    - size: Size of the vector

    Returns:
    - Random vector
    """
    return (np.random.rand(size) * 1000).astype(np.float32)

def check_equal(A1, A2):
    """
    Check if two vectors are equal within a specified tolerance.

    Parameters:
    - A1, A2: Vectors to compare

    Returns:
    - True if vectors are equal, False otherwise
    """
    return np.allclose(A1, A2, rtol=1e-5)

def main():
    # Set the vector size
    vector_size = 500000

    # Generate a random vector
    input_vector = generate_random_vector(vector_size)

    # Initialize output vectors
    output_vector_gpu = np.zeros_like(input_vector)
    output_vector_cpu = np.zeros_like(input_vector)

    ### ------------------- TO DO 1------------------- ###

    # Perform vector normalization on the GPU
    start_time = time.time()
    ### ------------------- TO DO 2------------------- ###
    gpu_time = time.time() - start_time

    ### ------------------- TO DO 3------------------- ###

    # Perform vector normalization on the CPU (naive implementation)
    start_time = time.time()
    naive_vector_normalize_cpu(input_vector, output_vector_cpu)
    cpu_time = time.time() - start_time

    # Check if the CPU and GPU results are equal
    if check_equal(output_vector_gpu, output_vector_cpu):
        print("CPU and GPU results are equal\n")
    else:
        print("Error! CPU and GPU results are different\n")

    # Print the execution times
    print(f"GPU Time: {gpu_time:.6f} seconds")
    print(f"CPU Time (naive): {cpu_time:.6f} seconds")

if __name__ == "__main__":
    main()